## Populate FASTianF1 RDF database

This notebook reports the main steps to download CSV files, process them and create an RDF dataset from them accordingly to an ontology.

To measure execution time in Jupyter notebooks: <code>pip install ipython-autotime</code>

In [1]:
# required libraries
import pandas as pd
import os
from pathlib import Path

In [2]:
# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# rdflib knows about some namespaces, like FOAF
from rdflib.namespace import FOAF, XSD

In [3]:
# CHECK DATE 
import datetime

# Paths

In [4]:
# parameters and URLs
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())
# print(path)
circuitsUrl = path + '\FASTianF1\data\DatasetF1\circuits.csv'
# print(circuitsUrl)
constructor_resultsUrl = path + '\FASTianF1\data\DatasetF1\constructor_results.csv'
constructor_standingsUrl = path + '\FASTianF1\data\DatasetF1\constructor_standings.csv'
constructorsUrl = path + '\FASTianF1\data\DatasetF1\constructors.csv'
driver_standingsUrl = path + '\FASTianF1\data\DatasetF1\driver_standings.csv'
driversUrl = path + '\FASTianF1\data\DatasetF1\drivers.csv'
lap_timesUrl = path + '\FASTianF1\data\DatasetF1\lap_times.csv'
pit_stopsUrl = path + '\FASTianF1\data\DatasetF1\pit_stops.csv'
qualifyingUrl = path + '\FASTianF1\data\DatasetF1\qualifying.csv'
racesUrl = path + '\FASTianF1\data\DatasetF1\\races.csv'
resultsUrl = path + '\FASTianF1\data\DatasetF1\\results.csv'
sprint_resultsUrl = path + '\FASTianF1\data\DatasetF1\sprint_results.csv'
statusUrl = path + '\FASTianF1\data\DatasetF1\status.csv'

# country codes
countriesURL = path + '\FASTianF1\data\countryCodes\wikipedia-iso-country-codes.csv'

# saving folder
savePath =  path + '\FASTianF1\data\\rdf\\'

# Namespaces

In [5]:
# Construct the country and the movie ontology namespaces not known by RDFlib
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
FO = Namespace("http://www.dei.unipd.it/database2/FASTianF1ontology#")

# Countries

In [6]:
#load the country codes
# we need to convert NaN values to something else otherwise NA strings are converted to NaN -> problem with Namibia
countries = pd.read_csv(countriesURL, sep=',', index_col='English short name lower case', keep_default_na=False, na_values=['_'])

# Drivers

In [7]:
# Load the CSV files in memory
drivers = pd.read_csv(driversUrl, sep=',', index_col='driverId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [8]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [9]:
%%time 
#measure execution time

#iterate over the movies dataframe
for index, row in drivers.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the movie id as URI
    Driver = URIRef(FO["driver"+str(index)])
    # Add triples using store's add() method.
    g.add((Driver, RDF.type, FO.Driver))
    g.add((Driver, FO['hasDriverRef'], Literal(row['driverRef'], datatype=XSD.string)))
    if(str(row['number']) != '\\N'):
        g.add((Driver, FO['hasNumber'], Literal(row['number'], datatype=XSD.integer)))
    if(str(row['code']) != '\\N'):
        g.add((Driver, FO['hasCode'], Literal(row['code'], datatype=XSD.string)))
    g.add((Driver, FO['hasForename'], Literal(row['forename'], datatype=XSD.string)))
    g.add((Driver, FO['hasSurname'], Literal(row['surname'], datatype=XSD.string)))
    #g.add((Driver, FO['hasDateOfBirth'], Literal(row['dob'], datatype=XSD.string)))
    g.add((Driver, FO['hasURL'], Literal(row['url'], datatype=XSD.string)))
    
    try:
        datetime.datetime.strptime(str(row['dob']), '%Y-%m-%d')
        g.add((Driver, FO['hasDateOfBirth'], Literal(row['dob'], datatype=XSD.date)))
    except ValueError:
        # probably it's the year alone
        # check length
        if (len(row['dob'])==4):
            #it is a year
            g.add((Driver, FO['hasDateOfBirth'], Literal(row['dob']+"-01-01", datatype=XSD.date)))

    ## handle country
    #there can be more than one country per movie
    for c in str(row['nationality']).split('-'):
        cName = c.strip()
        # check if the country exists
        # country.index == x returns an array of booleans, thus we need to use the any() method
        if((countries.index == cName).any() == True):
            #get the country code, convert to string and get the lower case to match the country codes in the ontology 
            code = str(countries[countries.index == cName]['Alpha-2 code'][0]).lower()
            # create the RDF node
            Country = URIRef(CNS[code])
            # add the edge connecting the Movie and the Country 
            g.add((Driver, FO['hasNationality'], Country))

CPU times: total: 234 ms
Wall time: 237 ms


In [10]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'drivers.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 203 ms
Wall time: 179 ms


# Circuits

In [11]:
# Load the CSV files in memory
circuits = pd.read_csv(circuitsUrl, sep=',', index_col='circuitId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [12]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [13]:
%%time 
#measure execution time

#iterate over the movies dataframe
for index, row in circuits.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the movie id as URI
    Circuit = URIRef(FO["circuit"+str(index)])
    Location = URIRef(FO["location"+str(row['location']).replace(" ", "")])
    # Add triples using store's add() method.
    g.add((Circuit, RDF.type, FO.Circuit))
    g.add((Circuit, FO['hasCircuitRef'], Literal(row['circuitRef'], datatype=XSD.string)))
    g.add((Circuit, FO['hasName'], Literal(row['name'], datatype=XSD.string)))
    g.add((Location, RDF.type, FO.Location))
    g.add((Circuit, FO['hasLocation'], Location))
    g.add((Circuit, FO['hasLat'], Literal(row['lat'], datatype=XSD.integer)))
    g.add((Circuit, FO['hasLng'], Literal(row['lng'], datatype=XSD.integer)))
    if(str(row['alt']) != '\\N'):
        g.add((Circuit, FO['hasAlt'], Literal(row['alt'], datatype=XSD.integer)))
    g.add((Circuit, FO['hasURL'], Literal(row['url'], datatype=XSD.string)))

    ## handle country
    cName = c.strip(str(row['country']))
    # check if the country exists
    # country.index == x returns an array of booleans, thus we need to use the any() method
    if((countries.index == cName).any() == True):
        #get the country code, convert to string and get the lower case to match the country codes in the ontology 
        code = str(countries[countries.index == cName]['Alpha-2 code'][0]).lower()
        # create the RDF node
        Country = URIRef(CNS[code])
        # add the edge connecting the Movie and the Country 
        g.add((Location, FO['hasCountry'], Country))

CPU times: total: 15.6 ms
Wall time: 25.9 ms


In [14]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'circuits.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 15.6 ms
Wall time: 18.9 ms


# Constructors

In [15]:
# Load the CSV files in memory
constructors = pd.read_csv(constructorsUrl, sep=',', index_col='constructorId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [16]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [17]:
%%time 
#measure execution time

#iterate over the movies dataframe
for index, row in constructors.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the movie id as URI
    Constructor = URIRef(FO["constructor"+str(index)])
    # Add triples using store's add() method.
    g.add((Constructor, RDF.type, FO.Constructor))
    g.add((Constructor, FO['hasConstructorRef'], Literal(row['constructorRef'], datatype=XSD.string)))
    g.add((Constructor, FO['hasName'], Literal(row['name'], datatype=XSD.string)))
    g.add((Constructor, FO['hasURL'], Literal(row['url'], datatype=XSD.string)))

    ## handle country
    #there can be more than one country per movie
    for c in str(row['nationality']).split('-'):
        cName = c.strip()
        # check if the country exists
        # country.index == x returns an array of booleans, thus we need to use the any() method
        if((countries.index == cName).any() == True):
            #get the country code, convert to string and get the lower case to match the country codes in the ontology 
            code = str(countries[countries.index == cName]['Alpha-2 code'][0]).lower()
            # create the RDF node
            Country = URIRef(CNS[code])
            # add the edge connecting the Movie and the Country 
            g.add((Constructor, FO['hasNationality'], Country))

CPU times: total: 46.9 ms
Wall time: 40.4 ms


In [18]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'constructors.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 15.6 ms
Wall time: 25.4 ms


# Status

In [19]:
# Load the CSV files in memory
status = pd.read_csv(statusUrl, sep=',', index_col='statusId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [20]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [21]:
%%time 
#measure execution time

#iterate over the movies dataframe
for index, row in status.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the movie id as URI
    Status = URIRef(FO["status"+str(index)])
    # Add triples using store's add() method.
    g.add((Status, RDF.type, FO.Status))
    g.add((Status, FO['hasName'], Literal(row['status'], datatype=XSD.string)))

CPU times: total: 15.6 ms
Wall time: 12 ms


In [22]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'status.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 15.6 ms
Wall time: 8.98 ms


# Race (and season)

In [23]:
# Load the CSV files in memory
races = pd.read_csv(racesUrl, sep=',', index_col='raceId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [24]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [25]:
%%time 
#measure execution time

#iterate over the movies dataframe
for index, row in races.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the movie id as URI
    Race = URIRef(FO["race"+str(index)])
    # Add triples using store's add() method.
    g.add((Race, RDF.type, FO.Race))
    g.add((Race, FO['hasRound'], Literal(row['round'], datatype=XSD.integer)))
    g.add((Race, FO['hasName'], Literal(row['name'], datatype=XSD.string)))
    g.add((Race, FO['hasRaceDate'], Literal(row['date'], datatype=XSD.date)))
    if(str(row['time']) != '\\N'):
        g.add((Race, FO['hasRaceTime'], Literal(row['time'], datatype=XSD.time)))
    g.add((Race, FO['hasURL'], Literal(row['url'], datatype=XSD.string)))
    
    if(str(row['fp1_date']) != '\\N'):
        g.add((Race, FO['hasFp1Date'], Literal(row['fp1_date'], datatype=XSD.date)))
    if(str(row['fp1_time']) != '\\N'):
        g.add((Race, FO['hasFp1Time'], Literal(row['fp1_time'], datatype=XSD.time)))
    if(str(row['fp2_date']) != '\\N'):
        g.add((Race, FO['hasFp2Date'], Literal(row['fp2_date'], datatype=XSD.date)))
    if(str(row['fp2_time']) != '\\N'):
        g.add((Race, FO['hasFp2Time'], Literal(row['fp2_time'], datatype=XSD.time)))
    if(str(row['fp3_date']) != '\\N'):
        g.add((Race, FO['hasFp3Date'], Literal(row['fp3_date'], datatype=XSD.date)))
    if(str(row['fp3_time']) != '\\N'):
        g.add((Race, FO['hasFp3Time'], Literal(row['fp3_time'], datatype=XSD.time)))
    
    Circuit = URIRef(FO["circuit"+str(row['circuitId'])])
    g.add((Race, FO['hasCircuit'], Circuit))
    
    Season = URIRef(FO["season"+str(row['year'])])
    g.add((Season, RDF.type, FO.Season))
    g.add((Season, FO['hasYear'], Literal(row['year'], datatype=XSD.integer)))
    g.add((Race, FO['inSeason'], Season))

CPU times: total: 281 ms
Wall time: 289 ms


In [26]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'race.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 219 ms
Wall time: 224 ms


# Race (partecipations)

In [27]:
# Load the CSV files in memory
r_partecipations = pd.read_csv(resultsUrl, sep=',', index_col='resultId')
driver_standings = pd.read_csv(driver_standingsUrl, sep=',', index_col="driverStandingsId")
joined = r_partecipations.merge(driver_standings, how='outer', on=['raceId','driverId'], suffixes=('', 'Total')).fillna('\\N')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [28]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [29]:
%%time 
#measure execution time

#iterate over the movies dataframe
for index, row in joined.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the movie id as URI
    R_partecipation = URIRef(FO["r_partecipation"+str(index)])
    #print(URIRef(str(index)))
    # Add triples using store's add() method.
    g.add((R_partecipation, RDF.type, FO.RacePartecipation))
    if(str(row['number']) != '\\N'):
        g.add((R_partecipation, FO['hasNumber'], Literal(row['number'], datatype=XSD.integer)))
    if(str(row['grid']) != '\\N'):
        g.add((R_partecipation, FO['hasGrid'], Literal(int(row['grid']), datatype=XSD.integer)))
    if(str(row['position']) != '\\N'):
        g.add((R_partecipation, FO['hasPosition'], Literal(row['position'], datatype=XSD.integer)))
    if(str(row['positionText']) != '\\N'):
        g.add((R_partecipation, FO['hasPositionText'], Literal(row['positionText'], datatype=XSD.string)))
    if(str(row['positionOrder']) != '\\N'):
        g.add((R_partecipation, FO['hasPositionOrder'], Literal(int(row['positionOrder']), datatype=XSD.integer)))
    if(str(row['points']) != '\\N'):
        g.add((R_partecipation, FO['hasPoints'], Literal(int(row['points']), datatype=XSD.integer)))
    if(str(row['laps']) != '\\N'):
        g.add((R_partecipation, FO['hasLaps'], Literal(int(row['laps']), datatype=XSD.integer)))
    if(str(row['time']) != '\\N'):
        g.add((R_partecipation, FO['hasTime'], Literal(row['time'], datatype=XSD.string)))
    if(str(row['milliseconds']) != '\\N'):
        g.add((R_partecipation, FO['hasMilliseconds'], Literal(row['milliseconds'], datatype=XSD.integer)))
    if(str(row['fastestLap']) != '\\N'):
        g.add((R_partecipation, FO['hasFastestLap'], Literal(row['fastestLap'], datatype=XSD.integer)))
    if(str(row['rank']) != '\\N'):
        g.add((R_partecipation, FO['hasFastestLapRank'], Literal(row['rank'], datatype=XSD.integer)))
    if(str(row['fastestLapTime']) != '\\N'):
        g.add((R_partecipation, FO['hasFastestLapTime'], Literal("00:0" + str(row['fastestLapTime']), datatype=XSD.time)))
    if(str(row['fastestLapSpeed']) != '\\N'):
        g.add((R_partecipation, FO['hasFastestLapSpeed'], Literal(row['fastestLapSpeed'], datatype=XSD.string)))
    if(str(row['pointsTotal']) != '\\N'):
        g.add((R_partecipation, FO['hasTotalPoints'], Literal(int(row['pointsTotal']), datatype=XSD.integer)))
    if(str(row['positionTotal']) != '\\N'):
        g.add((R_partecipation, FO['hasTotalPosition'], Literal(row['positionTotal'], datatype=XSD.integer)))
    if(str(row['positionTextTotal']) != '\\N'):
        g.add((R_partecipation, FO['hasTotalPositionText'], Literal(row['positionTextTotal'], datatype=XSD.string)))
    if(str(row['wins']) != '\\N'):
        g.add((R_partecipation, FO['hasDriverWins'], Literal(int(row['wins']), datatype=XSD.integer)))
    
    Driver = URIRef(FO["driver"+str(row['driverId'])])
    g.add((R_partecipation, FO['hasDriver'], Driver))
    
    if(str(row['constructorId']) != '\\N'):
        Constructor = URIRef(FO["constructor"+str(int(row['constructorId']))])
        g.add((R_partecipation, FO['hasConstructor'], Constructor))
    
    Race = URIRef(FO["race"+str(row['raceId'])])
    g.add((R_partecipation, FO['partecipatedInRace'], Race))
    
    if(str(row['statusId']) != '\\N'):
        Status = URIRef(FO["status"+str(int(row['statusId']))])
        g.add((R_partecipation, FO['hasStatus'], Status))

CPU times: total: 14.8 s
Wall time: 14.8 s


In [30]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'race_partecipations.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 12.8 s
Wall time: 12.8 s


# Qualifying

In [ ]:
# Load the CSV files in memory
q_partecipations = pd.read_csv(qualifyingUrl, sep=',', index_col='qualifyId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [ ]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [ ]:
%%time 
#measure execution time

#iterate over the movies dataframe
for index, row in q_partecipations.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the movie id as URI
    Q_partecipation = URIRef(FO["q_partecipation"+str(index)])
    # Add triples using store's add() method.
    g.add((Q_partecipation, RDF.type, FO.QualifPartecipation))
    g.add((Q_partecipation, FO['hasNumber'], Literal(row['number'], datatype=XSD.integer)))
    g.add((Q_partecipation, FO['hasPosition'], Literal(row['position'], datatype=XSD.integer)))
    g.add((Q_partecipation, FO['hasQ1Time'], Literal(row['q1'], datatype=XSD.time)))
    g.add((Q_partecipation, FO['hasQ2Time'], Literal(row['q2'], datatype=XSD.time)))
    g.add((Q_partecipation, FO['hasQ3Time'], Literal(row['q3'], datatype=XSD.time)))

    Driver = URIRef(FO["driver"+str(row['driverId'])])
    g.add((Q_partecipation, FO['hasDriver'], Driver))
    
    Constructor = URIRef(FO["constructor"+str(row['constructorId'])])
    g.add((Q_partecipation, FO['hasConstructor'], Constructor))
    
    Qualifying = URIRef(FO["qualifying"+str(row['raceId'])])
    g.add((Qualifying, RDF.type, FO.Qualifying))
    g.add((Q_partecipation, FO['partecipatedInQualif'], Qualifying))
    
    Q_date_time = races[races['raceId'] == row['raceId']]
    g.add((Qualifying, FO['hasQualiDate'], Literal(Q_date_time['quali_date'], datatype=XSD.date)))
    g.add((Qualifying, FO['hasQualiTime'], Literal(Q_date_time['quali_time'], datatype=XSD.time)))
    
    Race = URIRef(FO["race"+str(row['raceId'])])
    g.add((Race, FO['hasA'], Qualifying))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'qualifying.ttl', 'w') as file:
    file.write(g.serialize(format='turtle').decode("utf-8"))

# Sprint

In [ ]:
# Load the CSV files in memory
s_partecipations = pd.read_csv(sprint_resultsUrl, sep=',', index_col='resultId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [ ]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [ ]:
%%time 
#measure execution time

#iterate over the movies dataframe
for index, row in s_partecipations.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the movie id as URI
    S_partecipation = URIRef(FO["s_partecipation"+str(index)])
    # Add triples using store's add() method.
    g.add((S_partecipation, RDF.type, FO.QualifPartecipation))
    g.add((S_partecipation, FO['hasNumber'], Literal(row['number'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasGrid'], Literal(row['grid'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasPosition'], Literal(row['position'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasPositionText'], Literal(row['positionTest'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasPositionOrder'], Literal(row['positionOrder'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasPoints'], Literal(row['points'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasLaps'], Literal(row['laps'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasTime'], Literal(row['time'], datatype=XSD.time)))
    g.add((S_partecipation, FO['hasMilliseconds'], Literal(row['milliseconds'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasFastestLap'], Literal(row['fastestLap'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasFastestLapTime'], Literal(row['fastestLapTime'], datatype=XSD.time)))
    
    Driver = URIRef(FO["driver"+str(row['driverId'])])
    g.add((Q_partecipation, FO['hasDriver'], Driver))
    
    Constructor = URIRef(FO["constructor"+str(row['constructorId'])])
    g.add((Q_partecipation, FO['hasConstructor'], Constructor))
    
    Sprint = URIRef(FO["sprint"+str(row['raceId'])])
    g.add((Sprint, RDF.type, FO.Sprint))
    g.add((S_partecipation, FO['partecipatedInSprint'], Sprint))
    
    S_date_time = races[races['raceId'] == row['raceId']]
    g.add((Sprint, FO['hasSprintDate'], Literal(S_date_time['sprint_date'], datatype=XSD.date)))
    g.add((Sprint, FO['hasSprintTime'], Literal(S_date_time['sprint_time'], datatype=XSD.time)))
    
    Race = URIRef(FO["race"+str(row['raceId'])])
    g.add((Race, FO['hasA'], Sprint))
    
    Status = URIRef(FO["status"+str(row['statusId'])])
    g.add((S_partecipation, FO['hasStatus'], Status))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'sprint.ttl', 'w') as file:
    file.write(g.serialize(format='turtle').decode("utf-8"))